In [14]:
import pandas as pd
import numpy as np
migration = pd.read_csv('./data/migration_schedule.csv')
migration = migration.sort_values(by = ['Application'])
validation = pd.read_csv('./data/validation_tracker.csv')

In [15]:
migration

,Application,PLA,Iteration,Prod/Non-Prod,Non Prod,Rehost Migration/ Net New,Server count,P_Start Date,P_End Date,M_Start Date,Unnamed: 10
0,ACL Suites,"Jones, Melissa M",4.0,Production,NaN,Rehost,3,11/9/22,11/18/22,11/18/22,NaN
1,ADMI,"Temmel, Darryl J",NaN,Production,NaN,Net New,3,NaN,NaN,NaN,NaN
2,ADMI Screener,"Del Pilar, Jayson",6.0,Production,NaN,Rehost,2,12/13/22,12/16/22,12/16/22,NaN
3,Altimeter,"Jackson, Andy",6.0,Production,NaN,Rehost,2,12/13/22,12/16/22,12/16/22,NaN
4,Apollo,"Tanner, Todd A",1.0,Non-Production,Test,Rehost,4,10/20/22,10/27/22,10/27/22,NaN
...,...,...,...,...,...,...,...,...,...,...,...
83,WinAir,"Yeager, B Scott",5.0,Production,NaN,Rehost,1,12/5/22,12/9/22,12/9/22,NaN
15,eICU Intervention Tool,"Temmel, Darryl J",NaN,Production,NaN,Net New,0,NaN,NaN,NaN,NaN
20,eNICQ,Randy Overly,5.0,Production,NaN,Rehost,1,12/5/22,12/9/22,12/9/22,NaN
23,gCMS,"Wiehagen, Ryan J",NaN,Production,NaN,Net New,2,NaN,NaN,NaN,NaN


In [16]:
validation

,IT Cab,App ID,Application,Final Disposition,L1 Capability,Business Criticality,Emergency Tier,Data Manager,Data Custodian,Support Group,Iteration,Migration Schedule,PLA,Prod/NonProd
0,"Fisne, Joseph",APM0001678,ACL Suites,Rehost,Clinical,High,Non Critical,"Kerestus, Kevin J","Cameron, Crystal L",Admin Systems Group 2,NaN,NaN,"Jones, Melissa M",Prod
1,"Fisne, Joseph",APM0002399,ADMI,Net New,Clinical,Low,Non Critical,"Martin, Christa Lese","Trelease, Michael R",Software Development Support,NaN,NaN,"Temmel, Darryl J",Prod
2,"Fisne, Joseph",APM0002897,ADMI Screener,Rehost,Research,Low,Non Critical,"Martin, Christa Lese","Ditchey, Robert M",Research IT,NaN,NaN,"Del Pilar, Jayson",Prod
3,"Haines, S Theodore",APM0002985,Altimeter,Rehost,IT,Low,Non Critical,"Yancoskie, Matthew M","Seibert, Michael J",EUC - Enterprise Messaging and Collaboration S...,NaN,NaN,"Jackson, Andy",Prod
4,"Fisne, Joseph",APM0001854,Apollo,Rehost,Human Resources,Urgent,Non Critical,"Frye, Susan E","Seybert, James",HST Support 3,NaN,NaN,"Tanner, Todd A",Non-Prod
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,"Michaels, James G",APM0002284,Tower Web Viewer,Rehost,Corporate Services,Low,Non Critical,"Trafton, Kristina L","Wiehagen, Ryan J",GHP Sales and Marketing,NaN,NaN,"Reed, William J",Prod
74,"Michaels, James G",APM0001324,TransSend,Rehost,IT,Urgent,Non Critical,"Golimowski, John","Niedjaco, Felix C",GHP IT EDI Ops,NaN,NaN,"Wakeman, Bradley D",Non-Prod
75,"Moyer, Marlin",APM0001470,UPS Worldship - A,Rehost,Clinical,Low,Non Critical,"Zsido, Ronald T","Demcher, Gerard",HIT Enterprise Pharmacy – Outpatient,NaN,NaN,"Heisey, Brian E",Non-Prod
76,"Moyer, Marlin",APM0002560,WebDesk,Rehost,Clinical,Low,Non Critical,"Patel, Aalpen A","Moyer, Marlin E",HIT Radiology Support,NaN,NaN,"Crawford, Robert E",Prod


In [58]:
output = validation.copy(deep='True')
app_list = set(output['Application '].unique())

for index, row in migration.iterrows():
    app_name = row['Application']
    date = row['M_Start Date']
    
    prod = row['Prod/Non-Prod']
    if prod == 'Production':
        prod = 'Prod'
    else:
        prod = 'Non-Prod'
    
    data = output.loc[(output['Application '] == app_name) & (output['Prod/NonProd'] == prod)]
    index = data.index[0]
    
    if row['Rehost Migration/ Net New'] == 'Net New':
        output.at[index, 'Migration Schedule'] = 'Net New'
        
    elif (prod == 'Prod'):
        output.at[index, 'Migration Schedule'] = date
        
    else:
        env = row['Non Prod']

        value = output.iloc[index]['Migration Schedule']
        if str(value) == 'nan':
            value = ''
            
        if str(env) == 'nan':
            output.at[index, 'Migration Schedule'] = str(date)
            
        else:
            output.at[index, 'Migration Schedule'] = str(value) + str(env) + ' - ' + str(date) + ' '

    if app_name != output.iloc[index]['Application ']:
        print('Error:', app_name, '!=', output.iloc[index]['Application '])

In [59]:
cols = ['App - Migration', 'Dates - Migration', 'App - Validation', 'Dates - Validation', 'Prod/NonProd']
df = pd.DataFrame(columns=cols)
output = output.sort_values(by='Application ')

a, b, c, d, e = [], [], [], [], []
val_index = 0
app_val = ''
date_val = ''
for index, row in migration.iterrows():
    a.append(row['Application'])
    b.append(row['M_Start Date'])
    
    # assumes that app list is identical and not missing
    row_val = output.iloc[val_index]
    if row['Application'] != row_val['Application ']:
        c.append(app_val)
        d.append(date_val)
    else:
        val_index = val_index + 1
        app_val = row_val['Application ']
        date_val = row_val['Migration Schedule']
        c.append(app_val)
        d.append(date_val)
    
    e.append(row_val['Prod/NonProd'])

data= {'App - Migration':a, 'Dates - Migration':b, 'App - Validation':c, 'Dates - Validation':d, 'Prod/NonProd':e}
df = pd.DataFrame(data)
df.to_csv('diff.csv')

In [60]:
output.to_csv('output.csv')
output

,IT Cab,App ID,Application,Final Disposition,L1 Capability,Business Criticality,Emergency Tier,Data Manager,Data Custodian,Support Group,Iteration,Migration Schedule,PLA,Prod/NonProd
0,"Fisne, Joseph",APM0001678,ACL Suites,Rehost,Clinical,High,Non Critical,"Kerestus, Kevin J","Cameron, Crystal L",Admin Systems Group 2,NaN,11/18/22,"Jones, Melissa M",Prod
1,"Fisne, Joseph",APM0002399,ADMI,Net New,Clinical,Low,Non Critical,"Martin, Christa Lese","Trelease, Michael R",Software Development Support,NaN,Net New,"Temmel, Darryl J",Prod
2,"Fisne, Joseph",APM0002897,ADMI Screener,Rehost,Research,Low,Non Critical,"Martin, Christa Lese","Ditchey, Robert M",Research IT,NaN,12/16/22,"Del Pilar, Jayson",Prod
3,"Haines, S Theodore",APM0002985,Altimeter,Rehost,IT,Low,Non Critical,"Yancoskie, Matthew M","Seibert, Michael J",EUC - Enterprise Messaging and Collaboration S...,NaN,12/16/22,"Jackson, Andy",Prod
4,"Fisne, Joseph",APM0001854,Apollo,Rehost,Human Resources,Urgent,Non Critical,"Frye, Susan E","Seybert, James",HST Support 3,NaN,Test - 10/27/22,"Tanner, Todd A",Non-Prod
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,"Fisne, Joseph",APM0001948,WinAir,Rehost,Corporate Services,Urgent,Non Critical,"Nussbaum, Matthew D","Pursley, Edward E",HST Support 1,NaN,12/9/22,"Yeager, B Scott",Prod
15,"Fisne, Joseph",APM0001233,eICU Intervention Tool,Net New,Clinical,Medium,Non Critical,"Fletcher, David","Trelease, Michael R",Software Development Support,NaN,Net New,"Temmel, Darryl J",Prod
18,"Fisne, Joseph",APM0002196,eNICQ,Rehost,Clinical,0,Non Critical,"Gerrity, Gloria J","Seybert, James",HST Support 3,NaN,12/9/22,Randy Overly,Prod
21,"Michaels, James G",APM0001559,gCMS,Net New,Health Plan,Medium,Non Critical,"Sciandra, Joann","Wiehagen, Ryan J",GHP Application Support,NaN,Net New,"Wiehagen, Ryan J",Prod
